In [1]:
import pandas as pd
import requests
import re

In [6]:
url = f'https://api-extern.systembolaget.se/sb-api-ecommerce/v1/site/stores/24696138'
headers = {'ocp-apim-subscription-key': 'cfc702aed3094c86b92d6d4ff7a54c84' }
r = requests.get(url, headers=headers)

In [16]:
pd.set_option('max_columns',None)
df = pd.json_normalize(r.json()['storeStocks'])[['store.siteId', 'store.alias','store.isOpen','store.address','store.city','store.openingHours','store.position.latitude','store.position.longitude','stockBalance.stock']]